See README for goals and stuff.

This code is messy a.f., bad for ux, but can be polished.

Pulling code from:

- https://github.com/RobotLocomotion/drake/pull/14082
  - [`drake/bindings/pydrake/common/jupyter.py`](https://github.com/RobotLocomotion/drake/blob/d6cafe46a1eb16f6ef14b4c3a39295e3223c74bb/bindings/pydrake/common/jupyter.py#L79-L141)

In [ ]:
from contextlib import contextmanager
import functools
import time

from IPython.display import clear_output, display
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from packaging import version

In [ ]:
def _maybe_show_inline_matplotlib_plots():
    if version.parse(widgets.__version__) >= version.parse("7.0.0"):
        widgets.widgets.interaction.show_inline_matplotlib_plots()


@contextmanager
def interactive_update(out, *, capture_errors=False):
    """
    Provides a context that will clear an Output widget beforehand and then
    redirect output to the widget.
    Arguments:
        out (Output): The Output widget to which output should be directed.
        capture_errors: If True, will redirect errors to widgets. If False
            (Default), will capture the error and re-throw them. It is
            suggested to leave this at its default value.
    Note:
        This is a more generalized form of ``interactive_output``:
        https://github.com/jupyter-widgets/ipywidgets/blob/7.5.1/ipywidgets/widgets/interaction.py#L65-L85
    Warning:
        On ipywidgets<7.0.0, redirecting the output of matplotlib plots and
        widgets to Output widgets  is supported.
        However, on ipywidgets>=7.0.0, they appear to be. For an example of
        such, please see:
        https://github.com/RobotLocomotion/drake/pull/14082#pullrequestreview-490546550
    """  # noqa
    assert isinstance(out, widgets.Output), out
    error = None
    with out:
        clear_output(wait=True)
        if capture_errors:
            yield
            _maybe_show_inline_matplotlib_plots()
        else:
            try:
                yield
                _maybe_show_inline_matplotlib_plots()
            except Exception as e:
                error = e
    if error is not None:
        raise error


def decorate_interactive_update(out, *, capture_errors=False):
    """
    Decorates a function to be executed within a ``interactive_update`` context.
    See ``interactive_update`` for arguments.
    """
    assert isinstance(out, widgets.Output), out

    def decorator(f):

        @functools.wraps(f)
        def wrapped(*args, **kwargs):
            with interactive_update(out, capture_errors=capture_errors):
                return f(*args, **kwargs)

        return wrapped

    return decorator

In [ ]:
out = widgets.Output()
display(out)

In [ ]:
# Show that the output is synchronized across multiple output cells.
display(out)

In [ ]:
with out:
    print("Hello world")

In [ ]:
with out:
    print("Hello world x 2")

In [ ]:
# Direct context form.

for i in range(5):
    with interactive_update(out):
        print(f"Hello world x {i} {'.' * i}")
    time.sleep(0.2)

In [ ]:
@decorate_interactive_update(out)
def my_callback(i):
    print(f"Hello again x {i} {'.' * i}")

for i in range(5):
    my_callback(i)
    time.sleep(0.2)

In [ ]:
if version.parse(widgets.__version__) >= version.parse("7.0.0"):

    with interactive_update(out):
        display(widgets.Button(description="Widget 1"))

    time.sleep(0.2)

    with interactive_update(out):
        display(widgets.Checkbox(description="Widget 2"))

    time.sleep(0.2)

    with interactive_update(out):
        display(widgets.Dropdown(options=["Widget 3"]))

    time.sleep(0.2)

    with interactive_update(out):
        plt.plot(0, 0)

else:
    print("Skipping this section")

In [ ]:
display(out)

In [ ]:
class CustomError(Exception):
    pass

class Unreachable(Exception):
    pass

In [ ]:
# Show an error being thrown within a nominal context.

with interactive_update(out, capture_errors=True):
    raise CustomError()
print("Error got redirected :(")

In [ ]:
# Now show the error being redirected.

try:
    with interactive_update(out):
        raise CustomError()
    raise Unreachable()
except CustomError:
    print("Caught error directly - yay!")